# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [2]:
import main
import pandas as pd
import psycopg2 as pg

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

### film_in_category

In [21]:
film_in_cat = main.film_in_category(3)

film_in_cat

,title,languge,category
0,Backlash Undefeated,English,Children
1,Bear Graceland,English,Children
2,Beneath Rush,English,Children
3,Betrayed Rear,English,Children
4,Cabin Flash,English,Children
5,Casper Dragonfly,English,Children
6,Christmas Moonshine,English,Children
7,Circus Youth,English,Children
8,Clockwork Paradise,English,Children
9,Comancheros Enemy,English,Children


### number_films_in_category

In [22]:
number_film_in_cat = main.number_films_in_category(2)

number_film_in_cat

,category,count
0,Animation,66


### number_film_by_length

In [23]:
number_film_by_len = main.number_film_by_length(80, 100)

number_film_by_len

,length,count
0,80,10
1,81,3
2,82,7
3,83,6
4,84,13
5,85,17
6,86,5
7,87,6
8,88,4
9,89,6


### client_from_city

In [24]:
client_from_ci = main.client_from_city('Athenai')

client_from_ci

,city,first_name,last_name
0,Athenai,Linda,Williams


### avg_amount_by_length

In [25]:
avg_amount_by_len = main.avg_amount_by_length(120)

avg_amount_by_len

,length,avg
0,120,4.931176


### client_by_sum_length

In [26]:
client_by_sum_len = main.client_by_sum_length(4500)

client_by_sum_len

,first_name,last_name,sum
0,Diane,Collins,4514
1,Elizabeth,Brown,4531
2,Justin,Ngo,4550
3,Sherry,Marshall,4582
4,Marion,Snyder,4599
5,Russell,Brinson,4640
6,Marcia,Dean,4734
7,Wesley,Bull,4808
8,Clara,Shaw,4808
9,Tammy,Sanders,5065


### category_statistic_length

In [27]:
category_stat_len = main.category_statistic_length('Action')

category_stat_len

,category,avg,sum,min,max
0,Action,111.609375,7143,47,185


### Zadanie 1

Znajdź listę wszystkich filmów o takiej samej długości.

In [67]:
query_1 = '''
SELECT length, title
FROM film
WHERE length IN (SELECT length FROM film GROUP BY length HAVING COUNT(*) > 1)
ORDER BY length
'''

pd.read_sql(query_1, con=connection)

,length,title
0,46,Ridgemont Submarine
1,46,Iron Moon
2,46,Alien Center
3,46,Kwai Homeward
4,46,Labyrinth League
...,...,...
995,185,Darn Forrester
996,185,Sweet Brotherhood
997,185,Soldiers Evolution
998,185,Worst Banger


### Zadanie 2

Znajdź wszystkich klientów mieszkających w tym samym mieście.

In [107]:
query_2 = '''
SELECT city.city, customer.first_name, customer.last_name
FROM customer
JOIN address USING(address_id)
JOIN city USING(city_id)
WHERE city.city IN (
    SELECT city 
    FROM customer 
    JOIN address USING(address_id) 
    JOIN city USING(city_id) 
    GROUP BY city 
    HAVING COUNT(*) > 1)
ORDER BY city.city
'''

pd.read_sql(query_2, con=connection)

,city,first_name,last_name
0,Aurora,Scott,Shelley
1,Aurora,Clinton,Buford
2,London,Mattie,Hoffman
3,London,Cecil,Vines


### Zadanie 3

Oblicz średni koszt wypożyczenia wszystkich filmów.


In [115]:
query_3 = '''
SELECT AVG(payment.amount) AS average_price, film.title
FROM film
INNER JOIN inventory USING(film_id)
INNER JOIN rental USING(inventory_id)
INNER JOIN payment USING(rental_id)
GROUP BY film.title
'''

pd.read_sql(query_3, con=connection)


,average_price,title
0,6.323333,Graceland Dynamite
1,6.090000,Opus Ice
2,3.390000,Braveheart Human
3,5.865000,Wonderful Drop
4,2.918571,Rush Goodfellas
...,...,...
953,2.444545,Mockingbird Hollywood
954,1.990000,Gathering Calendar
955,1.913077,Drums Dynamite
956,4.240000,Samurai Lion


### Zadanie 4

Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.


In [33]:
query_4 = '''
SELECT DISTINCT category.name, COUNT(film_id)
FROM category
INNER JOIN film_category USING(category_id)
INNER JOIN film USING(film_id)
GROUP BY category.name
'''

pd.read_sql(query_4, con=connection)

,name,count
0,Games,61
1,Music,51
2,Classics,57
3,Children,60
4,Foreign,73
5,Family,69
6,Horror,56
7,Animation,66
8,New,63
9,Drama,62


### Zadanie 5

Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.


In [73]:
query_5 = '''
SELECT DISTINCT country.country, COUNT(customer_id)
FROM customer
INNER JOIN address USING(address_id)
INNER JOIN city USING(city_id)
INNER JOIN country USING(country_id)
GROUP BY country.country
ORDER BY country.country
'''

pd.read_sql(query_5, con=connection)

,country,count
0,Afghanistan,1
1,Algeria,3
2,American Samoa,1
3,Angola,2
4,Anguilla,1
...,...,...
103,Vietnam,6
104,"Virgin Islands, U.S.",1
105,Yemen,4
106,Yugoslavia,2


### Zadanie 6

Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.


In [106]:
query_6 = '''
SELECT store_id, manager_staff_id, address_id, last_update
FROM store
WHERE store_id = (
    SELECT store_id 
    FROM customer 
    GROUP BY store_id 
    HAVING COUNT(*) BETWEEN 100 AND 300)
'''

pd.read_sql(query_6, con=connection)

,store_id,manager_staff_id,address_id,last_update
0,2,2,2,2006-02-15 09:57:12


### Zadanie 7

Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.


In [79]:
query_7 = '''
SELECT customer.first_name, customer.last_name, SUM(film.length)
FROM customer
INNER JOIN rental USING(customer_id)
INNER JOIN inventory USING(inventory_id)
INNER JOIN film USING(film_id)
GROUP BY customer.last_name, customer.first_name
HAVING SUM(film.length) > 12000
'''

#przy 200 godzinach czyli 12000 minutach nie mamy żadnego klienta który osiągnął taki watchtime

pd.read_sql(query_7, con=connection)

,first_name,last_name,sum


In [41]:
#dla przykladu przy sprawdziwmy wachtime na poziomie 75 godzin

query_71 = '''
SELECT customer.first_name, customer.last_name, SUM(film.length) as SUM
FROM customer
INNER JOIN rental USING(customer_id)
INNER JOIN inventory USING(inventory_id)
INNER JOIN film USING(film_id)
GROUP BY customer.last_name, customer.first_name
HAVING SUM(film.length) > 4500
ORDER BY SUM DESC
'''

pd.read_sql(query_71, con=connection)

,first_name,last_name,sum
0,Karl,Seal,5388
1,Eleanor,Hunt,5360
2,Tammy,Sanders,5065
3,Wesley,Bull,4808
4,Clara,Shaw,4808
5,Marcia,Dean,4734
6,Russell,Brinson,4640
7,Marion,Snyder,4599
8,Sherry,Marshall,4582
9,Justin,Ngo,4550


Jak widać na powyższym przykładzie maksymalny watchtime wynosi 5388 minut.

### Zadanie 8

Oblicz średnią wartość wypożyczenia filmu.

In [42]:
query_8 = '''
SELECT AVG(amount)
FROM payment
'''

pd.read_sql(query_8, con=connection)

,avg
0,4.200606


### Zadanie 9

Oblicz średnią wartość długości filmu we wszystkich kategoriach.

In [43]:
query_9 = '''
SELECT category.name, AVG(film.length)
FROM category
INNER JOIN film_category USING(category_id)
INNER JOIN film USING(film_id)
GROUP BY category.name
'''

pd.read_sql(query_9, con=connection)

,name,avg
0,Sports,128.202703
1,Classics,111.666667
2,New,111.126984
3,Family,114.782609
4,Comedy,115.827586
5,Animation,111.015152
6,Travel,113.315789
7,Music,113.647059
8,Drama,120.838710
9,Horror,112.482143


### Zadanie 10

Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.

In [109]:
query_10 = '''
SELECT length(film.title) AS length, film.title AS title, ct.name
FROM category ct
JOIN film_category USING(category_id)
JOIN film USING(film_id)
GROUP BY ct.name, title
HAVING length(film.title) IN (
    SELECT max(length(film.title)) 
    FROM film 
    JOIN film_category USING(film_id) 
    JOIN category USING(category_id)
    WHERE category.name = ct.name)
ORDER BY ct.name
'''

pd.read_sql(query_10, con=connection)

,length,title,name
0,23,Entrapment Satisfaction,Action
1,22,Telemark Heartbreakers,Animation
2,20,Heartbreakers Bright,Children
3,20,Microcosmos Paradise,Children
4,20,Sweethearts Suspects,Children
5,23,Extraordinary Conquerer,Classics
6,23,Trainspotting Strangers,Comedy
7,22,Deliverance Mulholland,Documentary
8,22,Intolerable Intentions,Documentary
9,22,Goldfinger Sensibility,Drama


### Zadanie 11

Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [114]:
query_11 = '''
SELECT film.length AS max_length, cat.name, film.title
FROM film
JOIN film_category USING(film_id)
JOIN category cat USING(category_id)
GROUP BY cat.name, film.title, max_length
HAVING film.length IN (
    SELECT max(length)
    FROM film
    JOIN film_category USING(film_id)
    JOIN category USING(category_id)
    WHERE category.name = cat.name)
ORDER BY cat.name
'''

pd.read_sql(query_11, con=connection)

,max_length,name,title
0,185,Action,Darn Forrester
1,185,Action,Worst Banger
2,185,Animation,Gangs Pride
3,185,Animation,Pond Seattle
4,178,Children,Fury Murder
5,178,Children,Wrong Behavior
6,184,Classics,Conspiracy Spirit
7,185,Comedy,Control Anthem
8,183,Documentary,Wife Turn
9,183,Documentary,Young Language
